In [1]:
# Bilbiotecas para auxílio na programação matemática
import math, sys 
import numpy as np
import sympy as sp

from scipy import sparse # Produção das diagonais das matrizes
from scipy.sparse import diags 

# Plotagem 2D e 3D
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm


from os import path # Suficiente para manipulação de arquivos
    
# Para solução exata
from scipy.special import hermite
from math import factorial

%matplotlib inline
count = 0

# Para otimização dos sistemas
from scipy import optimize

import random

In [2]:
def somar(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] + B[i][j]

    return C

def sub(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] - B[i][j]

    return C

def dpsidt(t,psi, H):
    A = np.zeros((2,2), dtype=np.complex_)
    A = np.dot(complex(0,1),H)
    return -1*np.matmul(A,psi)

def rungeKutta(psi0, h, H, t = 2, t0 = 0, p = 0):
    
    n = (int)((t-t0))
    S = np.zeros((2,int(t)), dtype=np.complex_)
    r = 0
    
    for i in range(1 + p, n + 1 + p):
        
        S[0][r] = psi0[0][0]
        S[1][r] = psi0[1][0]
        
        k1 = dpsidt(t0, psi0, H)
        k2 = dpsidt(t0 + 0.5 * h, somar(psi0, np.dot((0.5*h), k1)), H)
        k3 = dpsidt(t0 + 0.5 * h, somar(psi0, np.dot((0.5*h), k2)), H)
        k4 = dpsidt(t0 + h, somar(psi0, np.dot(h, k3)), H)
        
        A = somar(np.dot(2,k3), k4)
        B = somar(np.dot(2,k2), k1)
        C = somar(A, B)
 
        psi0 = somar(psi0,np.dot((h / 6.0),(C)))
    
        t0 = t0 + h
        
        r = r + 1
        
    return S

def fo(D, M, x, h, H, psi_n, psi_o):
    
    H1 = [[0, x[0]],[x[0], 0]]
    H2 = [[0, x[1]],[x[1], 0]]
    
    N = rungeKutta(psi_n, h, somar(H,H1), t = 1000, t0 = 0, p = 0)
    O = rungeKutta(psi_o, h, somar(somar(H,H1),H2), t = 1000, t0 = 0, p = 0)

    return (np.linalg.norm(D-M)**2+np.linalg.norm(D-N)**2+np.linalg.norm(D-O)**2)

def foS(alpha, d, D, M, x, h, H, psi_n, psi_o):
    
    z = x + alpha * d
    
    H1 = [[0, z[0]],[z[0], 0]]
    H2 = [[0, z[1]],[z[1], 0]]
    
    N = rungeKutta(psi_n, h, somar(H,H1), t = 1000, t0 = 0, p = 0)
    O = rungeKutta(psi_o, h, somar(somar(H,H1),H2), t = 1000, t0 = 0, p = 0)

    return (np.linalg.norm(D-M)**2+np.linalg.norm(D-N)**2+np.linalg.norm(D-O)**2)

In [3]:
def SecaoAurea(d, D, M, x, h, H, psi_n, psi_o):
    eps = 0.0000001
    a = 0
    b = 1
    xa = b - 0.618*(b-a)
    xb = a + 0.618*(b-a)
    fxa = foS(xa, d, D, M, x, h, H, psi_n, psi_o)
    fxb = foS(xb, d, D, M, x, h, H, psi_n, psi_o)
    
    while(b - a > eps):
        if(fxa < fxb):
            b = xb
            xb = xa
            xa = b - 0.618*(b - a)
            fxb = fxa
            fxa = foS(xa, d, D, M, x, h, H, psi_n, psi_o)
        else:
            a = xa
            xa = xb
            xb = a + 0.618*(b - a)
            fxa = fxb
            fxb = foS(xb, d, D, M, x, h, H, psi_n, psi_o)
            
    xp = (a+b)/2
    return xp

#Avaliação do gradiente
def gradiente(x, h, D, M, H, psi_n, psi_o):
    h = 0.001
    grad = []
    
    for i in range(len(x)):
        xh = x.copy()
        xh[i] = xh[i] + h
        dx = (fo(D, M, xh, h, H, psi_n, psi_o)-fo(D, M, x, h, H, psi_n, psi_o))/h
        grad.append(dx)
    return grad

def MetodoGrad(x, h, D, M, H, psi_n, psi_o):
    
    eps = 0.01
    SequenciaPontos = [x]
    
    errox = eps + 1
    xant = x.copy()
    
    k = 0
    kmax = 100
    normagrad = eps + 1
    
    CP = ''
    
    while(CP == ''):
        d = -1*np.array(gradiente(x, h, D, M, H, psi_n, psi_o))
        #d = -1*(np.matmul(np.linalg.inv(hessiana(x, tempo, psi, psiExato)),gradiente(x, tempo, psi, psiExato)))
        alpha = SecaoAurea(d, D, M, x, h, H, psi_n, psi_o)
        x = x + alpha * d
        
        errox = np.linalg.norm(xant - x)
        xant = x.copy()
        if(errox <= eps):
            CP = CP + 'Erro x'
        normagrad = np.linalg.norm(d)
        if(normagrad <= eps):
            CP = CP + 'Norma Gradiente'
        if(k >= kmax):
            CP = 'Numero de iteracoes'
            
        k += 1
        SequenciaPontos.append(x)
        
    xOtimo = x
    return SequenciaPontos, k, xOtimo, CP

In [4]:
x = []
x.append(random.uniform(-0.50, 0.50))
x.append(random.uniform(-0.50, 0.50))

i = 0

psi0 = [[complex(0.80,0)],[complex(0.60,0)]]
h = 0.01

E = np.zeros((2,2))
I = np.zeros((2,2))
R = np.zeros((2,2))
H = E.astype(complex)

A = (1/2*np.pi)
B = (3/2*np.pi)

I = [[1, 0], [0, 1]]
R = [[1, 0], [0, 1]]
H = [[A, 0], [0, B]]

while i < 998:

    Result = np.zeros((2,3),dtype=np.complex_)

    Result = rungeKutta(psi0, h, H, t = 3, t0 = 0, p = i)

    D = np.zeros((2,1000),dtype=np.complex_)
    M = np.zeros((2,1000),dtype=np.complex_)

    psi_d = np.zeros((2,1), dtype=np.complex_)
    psi_m = np.zeros((2,1), dtype=np.complex_)
    psi_n = np.zeros((2,1), dtype=np.complex_)
    psi_o = np.zeros((2,1), dtype=np.complex_)

    psi_d = [[1/np.sqrt(2)],[1/np.sqrt(2)]]
    psi_m = [[Result[0][0]],[Result[1][0]]]
    psi_n = [[Result[0][1]],[Result[1][1]]]
    psi_o = [[Result[0][2]],[Result[1][2]]]

    D = rungeKutta(psi_d, h, H, t = 1000, t0 = 0, p = 0)
    M = rungeKutta(psi_m, h, H, t = 1000, t0 = 0, p = 0)

    resultado = MetodoGrad(x, h, D, M, H, psi_n, psi_o)
    print('iteracoes = ', resultado[1])
    print('x = ', resultado[2])
    print('Criterio de parada: ', resultado[3])
    
    x = []
    u = np.zeros((2,1))
    x.append(random.uniform(-10, 10))
    x.append(random.uniform(-10, 10))
    
    u = [[0, resultado[2][0]],[resultado[2][0], 0]]
        
    i = i + 1
    H = somar(H, u)
    
    psi_t = np.zeros((2,2), dtype=np.complex_)
    
    psi_t = rungeKutta(psi0, h, H, t = 2, t0 = 0, p = 0)
    
    psi0[0][0] = psi_t[0][1]
    psi0[1][0] = psi_t[1][1]
    
    print(psi0)

iteracoes =  2
x =  [-74.28332936 -29.00184577]
Criterio de parada:  Erro x
[[(0.6021234364988992+0.3976846613679696j)], [(0.4584700083403108+0.5170995902627495j)]]
iteracoes =  2
x =  [-2.66832667 -6.08280805]
Criterio de parada:  Erro x
[[(0.08735580856425629+0.6100108350157827j)], [(0.08521663682369313+0.7804950440275213j)]]
iteracoes =  1
x =  [ 0.92662807 -2.0576888 ]
Criterio de parada:  Erro x
[[(-0.4657642234999747+0.5168455753455719j)], [(-0.32675592693374733+0.635125359274745j)]]
iteracoes =  5
x =  [-1.80774126  5.66090889]
Criterio de parada:  Erro x
[[(-0.7791245901020911+0.15726390234910054j)], [(-0.5812987253170987+0.14929138925934943j)]]
iteracoes =  3
x =  [6.15673369 7.7226094 ]
Criterio de parada:  Erro x
[[(-0.6958749931527164-0.2520899412210251j)], [(-0.5512506342192621-0.3727378710938621j)]]
iteracoes =  6
x =  [5.62512568 1.31459886]
Criterio de parada:  Erro x
[[(-0.33452952414620984-0.5367391790571162j)], [(-0.3085003405670509-0.7032840377942617j)]]
iteracoes =

KeyboardInterrupt: 